# Turing patterns and your own automata (16 points)

Animal skin patterns are a beautiful and intriguing example of pattern formation in biology. They were studied by famous English mathematician Alan Turing, who, in 1952, published a paper titled *The Chemical Basis of Morphogenesis*. In his model, Turing described interactions between two homogeneneously distributed substances, that produce stable patterns. His model used partial differential equations. During this classes you'll implement a simpler, CA version of this model developed by [David Young](https://www.sciencedirect.com/science/article/abs/pii/0025556484900609).
![image](https://upload.wikimedia.org/wikipedia/commons/a/a1/TuringPattern.PNG)
<center>(source: <a href="https://upload.wikimedia.org/wikipedia/commons/a/a1/TuringPattern.PNG">wikimedia.org</a>)</center>

## David Young model

##### Grid
Two-dimensional rectangular grid.

##### Cells
Binary cells: `0` (inactive/passive) or `1` (active).

##### Neighbourhood
This model uses two Moore neighbourhoods. The first, with radius $R_a$, is used for determining the number of active cells (*short-range activation*). The second one, with greater radius $R_i$ is used for counting the amount of inactive/passive cells (*long-range inhibition*). $R_a$ and $R_i$ ($R_a<R_i$), as well as $w_a$ and $w_i$, are the model's parameters. We're considering only odd $R_a$ and $R_i$ values.

##### Model
Mathematically speaking, the model can be described as (Hiroki Sayama, *Introduction to the Modeling and Analysis of Complex Systems*):
* <center>Short-range neighbourhood:</center>
$$N_a = \left\{ x'\left| |x'|\right. \leq R_a \right\}$$
* <center>Long-range neighbourhood:</center>
 $$N_i = \left\{ x'\left| |x'|\right. \leq R_i \right\}$$
* <center>Transition function:</center>
 $$a_t(x) = w_a\sum_{x' \in N_a}s_t(x+x')-w_i\sum_{x' \in N_i}s_t(x+x')$$
* $s_t$ is a mathematical function that maps location to state, the sum is simply a sum of neighbours values
  $$s_{t+1}(x) = \left\{ \begin{array}{ll}1 & \text{if } a_t(x) > 0\\0 & \text{otherwise}\end{array}\right.$$


## Python implementation of David Young's model (4 points)
Fill in the gaps in the following code.

In [71]:
import numpy as np
import itertools
import pygame

from pygame.locals import (
    K_UP,
    K_DOWN,
    K_LEFT,
    K_RIGHT,
    K_ESCAPE,
    KEYDOWN,
    QUIT,
)

# Grid size
rows = 80
cols = 80

In [72]:
class Model:
    def __init__(self, rows, cols, Ra, Ri, wa, wi, p1):
        self.grid = np.random.choice([0, 1], size=(rows, cols), p=[1-p1, p1])
        self.p1 = p1
        self.Ra = Ra
        self.Ri = Ri
        self.wa = wa
        self.wi = wi

    def calculate_neighbourhood_by_radius(self, x, y, r, count_what):
        y_indices, x_indices = np.ogrid[-r:r+1, -r:r+1]
        mask = x_indices**2 + y_indices**2 <= r**2
        x_min, x_max = max(0, x-r), min(self.grid.shape[0], x+r+1)
        y_min, y_max = max(0, y-r), min(self.grid.shape[1], y+r+1)
        neighbourhood = self.grid[x_min:x_max, y_min:y_max]
        count = np.sum(neighbourhood[mask[:neighbourhood.shape[0], :neighbourhood.shape[1]]] == count_what)
        if self.grid[x, y] == count_what:
            count -= 1
        return count
    
    def calculate_neighbourhood_by_square(self, x, y, r, count_what):
        x_min, x_max = max(0, x-r), min(self.grid.shape[0], x+r+1)
        y_min, y_max = max(0, y-r), min(self.grid.shape[1], y+r+1)
        if self.grid[x, y] == count_what:
            return np.sum(self.grid[x_min:x_max, y_min:y_max] == count_what) - 1
        return np.sum(self.grid[x_min:x_max, y_min:y_max] == count_what)
    
    def update(self):
        rows, cols = self.grid.shape
        new_grid = self.grid.copy()
        for x in range(rows):
            for y in range(cols):
                ar_neighbourhood_val = self.calculate_neighbourhood_by_radius(x, y, self.Ra, 1)
                ir_neighbourhood_val = self.calculate_neighbourhood_by_radius(x, y, self.Ri, 1)
                # ar_neighbourhood_val = self.calculate_neighbourhood_by_square(x, y, self.Ra, 1)
                # ir_neighbourhood_val = self.calculate_neighbourhood_by_square(x, y, self.Ri, 1)
                a = ar_neighbourhood_val * self.wa - ir_neighbourhood_val * self.wi
                if a > 0:
                    new_grid[x, y] = 1
                else:
                    new_grid[x, y] = 0
        self.grid = new_grid

    def drawGrid(self, screen, w_width, w_height, padding_y = 0):
        black = (0, 0, 0)
        white = (255, 255, 255)
        rows, cols = self.grid.shape
        blockSize = min(w_width // cols, (w_height-padding_y) // rows)
        for x in range(rows):
            for y in range(cols):
                pos_x = blockSize * x
                pos_y = blockSize * y
                rect = pygame.Rect(pos_x, pos_y+padding_y, blockSize, blockSize)
                if self.grid[x, y] == 1:
                    pygame.draw.rect(screen, black, rect, 0)
                else:
                    pygame.draw.rect(screen, white, rect, 0)

In [73]:
class Simulation():
    def __init__(self, model, timerEnabled = False, w_width = 800, w_height = 800):
        pygame.init()
        self.w_width = w_width
        self.w_height = w_height
        self.screen = pygame.display.set_mode([self.w_width, self.w_height])
        self.model = model
        self.time_delay = 100 # 0.1 s
        self.timer_event = pygame.USEREVENT + 1
        self.running = False
        self.timerEnabled = timerEnabled
        pygame.time.set_timer(self.timer_event, self.time_delay)

    def draw(self):
        self.screen.fill((128, 128, 128))
        self.model.drawGrid(self.screen, self.w_width, self.w_height)
        pygame.display.flip()

    def run(self):
        self.running = True
        self.draw()
        while self.running:
            for event in pygame.event.get():   
                if event.type == QUIT:
                    self.running = False
                if event.type == KEYDOWN:
                    self.model.update()
                    self.draw()
                if self.timerEnabled and event.type == self.timer_event:
                    self.model.update()
                    self.draw()
        pygame.quit()

In [ ]:
default_model = Model(rows,cols, 3, 5, 1 , 0.2, 0.5)
sim = Simulation(default_model)
sim.run()

## Analysis (5 points)
Check different values of $w_a, w_i, R_a, R_b$. Find 5 visually different stable or oscilating patterns. Write down the parameteres and present results. Two sets of parameteres are considered to be different if at least two of the parameters are **substantially** different.

In [74]:
stable_model = Model(rows,cols, 3, 5, 2.77, 1, 0.5)
sim = Simulation(stable_model)
sim.run()

KeyboardInterrupt: 

In [75]:
growing_out_of_squares_model = Model(rows,cols, 3, 9, 0.5, 0.11, 1)
sim = Simulation(growing_out_of_squares_model)
sim.run()

KeyboardInterrupt: 

In [76]:
oscilating_model = Model(rows,cols, 3, 5, 0.5, 0.15, 0.5)
sim = Simulation(oscilating_model)
sim.run()

KeyboardInterrupt: 

In [77]:
center_shape_model = Model(rows,cols, 9, 23, 0.3, 0.05, 0.9)
sim = Simulation(center_shape_model)
sim.run()

KeyboardInterrupt: 

## Improvements (3 points)
In order to get this points you have to complete all of the previous tasks. Add widgets for parameters changing (e.g. slider or text field)


In [78]:
import pygame
import numpy as np

class Slider:
    def __init__(self, x, y, w, h, min_val, max_val, initial_val, label):
        self.rect = pygame.Rect(x, y, w, h)
        self.min_val = min_val
        self.max_val = max_val
        self.value = initial_val
        self.label = label
        self.dragging = False

    def draw(self, screen):
        pygame.draw.rect(screen, (200, 200, 200), self.rect)
        pygame.draw.rect(screen, (100, 100, 100), (self.rect.x, self.rect.y + self.rect.h // 2 - 2, self.rect.w, 4))
        handle_x = self.rect.x + int((self.value - self.min_val) / (self.max_val - self.min_val) * self.rect.w)
        pygame.draw.circle(screen, (0, 0, 0), (handle_x, self.rect.y + self.rect.h // 2), self.rect.h // 2)
        font = pygame.font.SysFont(None, 24)
        label_surf = font.render(f'{self.label}: {self.value:.2f}', True, (0, 0, 0))
        screen.blit(label_surf, (self.rect.x - 70, self.rect.y))

    def handle_event(self, event):
        if event.type == pygame.MOUSEBUTTONDOWN:
            if self.rect.collidepoint(event.pos):
                self.dragging = True
        elif event.type == pygame.MOUSEBUTTONUP:
            self.dragging = False
        elif event.type == pygame.MOUSEMOTION:
            if self.dragging:
                rel_x = event.pos[0] - self.rect.x
                rel_x = max(0, min(self.rect.w, rel_x))
                self.value = self.min_val + (self.max_val - self.min_val) * (rel_x / self.rect.w)

class SimulationCustomizable(Simulation):
    def __init__(self, model, timerEnabled=False):
        super().__init__(model, timerEnabled, 800, 900)
        self.wa_slider = Slider(90, 10, 550, 20, 0, 10, model.wa, 'w_a')
        self.wi_slider = Slider(90, 40, 550, 20, 0, 10, model.wi, 'w_i')
        self.p1_slider = Slider(90, 70, 550, 20, 0, 1, model.p1, 'p')
        self.padding_y = 100

    def draw(self):
        self.screen.fill((128, 128, 128))
        self.model.drawGrid(self.screen, self.w_width, self.w_height, self.padding_y)
        self.wa_slider.draw(self.screen)
        self.wi_slider.draw(self.screen)
        self.p1_slider.draw(self.screen)
        pygame.display.flip()

    def run(self):
        self.running = True
        self.draw()
        while self.running:
            for event in pygame.event.get():
                if event.type == QUIT:
                    self.running = False
                if event.type == KEYDOWN and event.key == pygame.K_SPACE:
                    self.model.update()
                    self.draw()
                if event.type == pygame.KEYDOWN and event.key == pygame.K_r:
                    self.model.grid = np.random.choice([0, 1], size=(rows, cols), p=[1-self.model.p1, self.model.p1])
                    self.draw()
                if self.timerEnabled and event.type == self.timer_event:
                    self.model.update()
                    self.draw()
                self.wa_slider.handle_event(event)
                self.wi_slider.handle_event(event)
                self.p1_slider.handle_event(event)
            self.model.wa = self.wa_slider.value
            self.model.wi = self.wi_slider.value
            self.model.p1 = self.p1_slider.value
            self.draw()
        pygame.quit()

In [79]:
stable_model = Model(rows,cols, 3, 5, 2.77, 1, 0.5)
custom_sim = SimulationCustomizable(stable_model)
custom_sim.run()

KeyboardInterrupt: 

# Your cellular automata (4 points)

In order to get this points you have to complete all of the previous tasks.
Find another example of CA based model, descirbe the rules and implement the automata

In [80]:
class ForestFireModel:
    def __init__(self, rows, cols, p_tree, p_fire, p_empty):
        self.grid = np.random.choice([0, 1, 2], size=(rows, cols), p=[p_empty, p_tree, p_fire])
        self.p_tree = p_tree
        self.p_fire = p_fire
        self.p_empty = p_empty

    def update(self):
        rows, cols = self.grid.shape
        new_grid = self.grid.copy()
        for x in range(rows):
            for y in range(cols):
                if self.grid[x, y] == 2: 
                    new_grid[x, y] = 0  
                elif self.grid[x, y] == 1: 
                    if 2 in self.grid[max(0, x-1):min(rows, x+2), max(0, y-1):min(cols, y+2)]:
                        new_grid[x, y] = 2 
                    elif np.random.random() < self.p_fire:
                        new_grid[x, y] = 2  
                elif self.grid[x, y] == 0: 
                    if np.random.random() < self.p_tree:
                        new_grid[x, y] = 1
        self.grid = new_grid

    def drawGrid(self, screen, w_width, w_height, padding_y=0):
        green = (0, 255, 0)
        red = (255, 0, 0)
        black = (0, 0, 0)
        rows, cols = self.grid.shape
        blockSize = min(w_width // cols, (w_height - padding_y) // rows)
        for x in range(rows):
            for y in range(cols):
                pos_x = blockSize * x
                pos_y = blockSize * y
                rect = pygame.Rect(pos_x, pos_y + padding_y, blockSize, blockSize)
                if self.grid[x, y] == 1:
                    pygame.draw.rect(screen, green, rect, 0)
                elif self.grid[x, y] == 2:
                    pygame.draw.rect(screen, red, rect, 0)
                else:
                    pygame.draw.rect(screen, black, rect, 0)


In [81]:
forest_fire_model = ForestFireModel(rows, cols, 0.05, 0.05, 0.9)
forest_fire_sim = Simulation(forest_fire_model)
forest_fire_sim.run()

KeyboardInterrupt: 